In [ ]:
import h5py
import matplotlib.colors as cols
from apt_importers import *

In [ ]:
h5file = h5py.File('Data/h5py_files/R12_Al-Sc.h5', 'r')
visu = h5file['Visualization']
visu_keys = visu.keys()
dpos = pd.concat([pd.Series(visu[k].value, name=k) for k in visu_keys], axis=1)
volvis(dpos)
obb = OBB.build_from_points(dpos.loc[:, ['x', 'y', 'z']].values)

In [ ]:
Sc_pos = dpos.loc[dpos.Element == 'Sc', :]
pos1, pos2, _ = singleton_removal(Sc_pos, k=10, alpha=0.01)

viz = False
if viz is True:
    cm = plt.get_cmap('gist_rainbow')
    cmap = np.asarray([cm(1. * i / 2) for i in range(2)])
    colors = [np.tile(cmap[0, :], (len(pos1), 1)), np.tile(cmap[1, :], (len(pos2), 1))]

    canvas = multiple_volvis([pos1, pos2], colors=colors, obb=obb)

In [ ]:
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.preprocessing import normalize

cpos = pos1.loc[:, ['x', 'y', 'z']].values
A_Sc = rbf_kernel(cpos, gamma=1./20.)
A_Sc[A_Sc < 1e-3] = 0

In [ ]:
import networkx as nx
import matplotlib.colors as cols

A_Sc_norm = normalize(A_Sc, norm='l1', axis=1)
A_Sc_norm[A_Sc_norm < 1e-3] = 0

G = nx.from_numpy_array(A_Sc)
G_Sc = nx.from_numpy_array(A_Sc_norm)
elarge=[(u,v) for (u,v,d) in G_Sc.edges(data=True) if d['weight'] < 0.04]

pos = nx.spring_layout(G, k=0.2)
cmap = plt.get_cmap('gist_rainbow')

degree = np.sum(A_Sc, axis=1)
degree = degree / np.max(degree)

fig = plt.figure()
ax = fig.add_subplot(111)
nodes = nx.draw_networkx_nodes(G, pos, node_color=degree, edgecolors='k', ax=ax)
cbar = plt.colorbar(nodes)
cbar.ax.tick_params(labelsize=26) 
nx.draw_networkx_edges(G, pos, edgelist=G_Sc.edges(data=True), width=1, ax=ax,style='dashed')
ax.set_axis_off()
plt.savefig('/home/larpanmuk/Documents/Papers_n_Thesis/APT_ML/fig/graph.png', bbox_inches='tight', dpi=300)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
img = ax.imshow(A_Sc, cmap='jet')
ax.set_xlabel('Scandium Atoms')
ax.set_ylabel('Scandium Atoms')
ax.xaxis.label.set_size(26)
ax.yaxis.label.set_size(26)           
cbar = plt.colorbar(img)
ax.set_yticks([])
ax.set_xticks([])
cbar.ax.tick_params(labelsize=26) 
plt.savefig('/home/larpanmuk/Documents/Papers_n_Thesis/APT_ML/fig/distance.png', bbox_inches='tight', dpi=300)

In [ ]:
import community

partition = community.best_partition(G_Sc)
comm_str = np.asarray([val for (key, val) in partition.iteritems()])
pos2 = dpos.loc[dpos.element != 'Sc']

In [ ]:
convex_hull_clusters(pos=pos1, comm_str=comm_str, pos2=pos2, viz=True, obb=obb, same_color=True)